## Assignment 1: Multilingual Embedding-based Machine Translation 

## 1 Task) 13 points

**In this homework** **<font color='red'>YOU</font>** will make machine translation system without using parallel corpora, alignment, attention, 100500 depth super-cool recurrent neural network and all that kind superstuff.

But even without parallel corpora this system can be good enough (hopefully). 

For our system we choose two kindred Slavic languages: Ukrainian and Russian. 

# PLEASE CHECK ASSERT, BE SURE THAT YOU IMPLEMENTED ALL THE CODE CORRECT IN FIRST PART

### Feel the difference!

(_синій кіт_ vs. _синій кит_)

![blue_cat_blue_whale.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/blue_cat_blue_whale.png)

### Fragment of the Swadesh list for some slavic languages

The Swadesh list is a lexicostatistical stuff. It's named after American linguist Morris Swadesh and contains basic lexis. This list are used to define subgroupings of languages, its relatedness.

So we can see some kind of word invariance for different Slavic languages.


| Russian         | Belorussian              | Ukrainian               | Polish             | Czech                         | Bulgarian            |
|-----------------|--------------------------|-------------------------|--------------------|-------------------------------|-----------------------|
| женщина         | жанчына, кабета, баба    | жінка                   | kobieta            | žena                          | жена                  |
| мужчина         | мужчына                  | чоловік, мужчина        | mężczyzna          | muž                           | мъж                   |
| человек         | чалавек                  | людина, чоловік         | człowiek           | člověk                        | човек                 |
| ребёнок, дитя   | дзіця, дзіцёнак, немаўля | дитина, дитя            | dziecko            | dítě                          | дете                  |
| жена            | жонка                    | дружина, жінка          | żona               | žena, manželka, choť          | съпруга, жена         |
| муж             | муж, гаспадар            | чоловiк, муж            | mąż                | muž, manžel, choť             | съпруг, мъж           |
| мать, мама      | маці, матка              | мати, матір, неня, мама | matka              | matka, máma, 'стар.' mateř    | майка                 |
| отец, тятя      | бацька, тата             | батько, тато, татусь    | ojciec             | otec                          | баща, татко           |
| много           | шмат, багата             | багато                  | wiele              | mnoho, hodně                  | много                 |
| несколько       | некалькі, колькі         | декілька, кілька        | kilka              | několik, pár, trocha          | няколко               |
| другой, иной    | іншы                     | інший                   | inny               | druhý, jiný                   | друг                  |
| зверь, животное | жывёла, звер, істота     | тварина, звір           | zwierzę            | zvíře                         | животно               |
| рыба            | рыба                     | риба                    | ryba               | ryba                          | риба                  |
| птица           | птушка                   | птах, птиця             | ptak               | pták                          | птица                 |
| собака, пёс     | сабака                   | собака, пес             | pies               | pes                           | куче, пес             |
| вошь            | вош                      | воша                    | wesz               | veš                           | въшка                 |
| змея, гад       | змяя                     | змія, гад               | wąż                | had                           | змия                  |
| червь, червяк   | чарвяк                   | хробак, черв'як         | robak              | červ                          | червей                |
| дерево          | дрэва                    | дерево                  | drzewo             | strom, dřevo                  | дърво                 |
| лес             | лес                      | ліс                     | las                | les                           | гора, лес             |
| палка           | кій, палка               | палиця                  | patyk, pręt, pałka | hůl, klacek, prut, kůl, pálka | палка, пръчка, бастун |

But the context distribution of these languages demonstrates even more invariance. And we can use this fact for our for our purposes.

## Data

In [1]:
import gensim
import numpy as np
from gensim.models import KeyedVectors

Download embeddings here:
* [cc.uk.300.vec.zip](https://www.dropbox.com/scl/fo/czrdhfndzvirs0c3x4adh/AIDkoaHAyd3bcpAegJ4bz3A?rlkey=f7xnl9lqiahjbor3ucxyy6u6p&st=ee70k4ik&dl=0)
* [cc.ru.300.vec.zip](https://www.dropbox.com/scl/fo/czrdhfndzvirs0c3x4adh/AIDkoaHAyd3bcpAegJ4bz3A?rlkey=f7xnl9lqiahjbor3ucxyy6u6p&st=ee70k4ik&dl=0)

Load embeddings for ukrainian and russian.

In [2]:
uk_emb = KeyedVectors.load_word2vec_format(r"cc.uk.300.vec")

In [3]:
ru_emb = KeyedVectors.load_word2vec_format(r"cc.ru.300.vec")

In [4]:
ru_emb.most_similar([ru_emb["август"]], topn=10)

[('август', 1.0000001192092896),
 ('июль', 0.9383152723312378),
 ('сентябрь', 0.9240029454231262),
 ('июнь', 0.9222574830055237),
 ('октябрь', 0.9095539450645447),
 ('ноябрь', 0.8930036425590515),
 ('апрель', 0.8729087114334106),
 ('декабрь', 0.8652557730674744),
 ('март', 0.8545795679092407),
 ('февраль', 0.8401415944099426)]

In [5]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 0.9999998807907104),
 ('липень', 0.9096441268920898),
 ('вересень', 0.9016969203948975),
 ('червень', 0.8992518782615662),
 ('жовтень', 0.8810408115386963),
 ('листопад', 0.8787633180618286),
 ('квітень', 0.8592804670333862),
 ('грудень', 0.8586863279342651),
 ('травень', 0.840811014175415),
 ('лютий', 0.8256431221961975)]

Load small dictionaries for correspoinding words pairs as trainset and testset.

In [6]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r", encoding="utf-8") as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [7]:
uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")

In [8]:
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")

## Embedding space mapping

Let $x_i \in \mathrm{R}^d$ be the distributed representation of word $i$ in the source language, and $y_i \in \mathrm{R}^d$ is the vector representation of its translation. Our purpose is to learn such linear transform $W$ that minimizes euclidian distance between $Wx_i$ and $y_i$ for some subset of word embeddings. Thus we can formulate so-called Procrustes problem:

$$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$$
or
$$W^*= \arg\min_W ||WX - Y||_F$$

where $||*||_F$ - Frobenius norm.

In Greek mythology, Procrustes or "the stretcher" was a rogue smith and bandit from Attica who attacked people by stretching them or cutting off their legs, so as to force them to fit the size of an iron bed. We make same bad things with source embedding space. Our Procrustean bed is target embedding space.

![embedding_mapping.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/embedding_mapping.png)

![procrustes.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/procrustes.png)

But wait...$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$ looks like simple multiple linear regression (without intercept fit). So let's code.

Let's take a look at neigbours of the vector of word _"серпень"_ (_"август"_ in Russian) after linear transform.

In [9]:
from sklearn.linear_model import LinearRegression

mapping = LinearRegression(fit_intercept=False)

mapping.fit(X_train, Y_train)

LinearRegression(fit_intercept=False)

In [10]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.8541286587715149),
 ('июнь', 0.8411202430725098),
 ('март', 0.839699387550354),
 ('сентябрь', 0.8359869718551636),
 ('февраль', 0.8329297304153442),
 ('октябрь', 0.8311845660209656),
 ('ноябрь', 0.8278924226760864),
 ('июль', 0.8234530091285706),
 ('август', 0.8120501637458801),
 ('декабрь', 0.8039003610610962)]

We can see that neighbourhood of this embedding cosists of different months, but right variant is on the ninth place.

As quality measure we will use precision top-1, top-5 and top-10 (for each transformed Ukrainian embedding we count how many right target pairs are found in top N nearest neighbours in Russian embedding space).

In [11]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        similar_words = ru_emb.most_similar(mapped_vectors[i].reshape(1, -1), topn=topn)
        for word, _ in similar_words:
            if word == ru:
                num_matches += 1
                break  
    precision_val = num_matches / len(pairs)
    return precision_val

In [12]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [13]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

In [14]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

assert precision_top1 >= 0.635
assert precision_top5 >= 0.813

## Making it better (orthogonal Procrustean problem)

It can be shown (see original paper) that a self-consistent linear mapping between semantic spaces should be orthogonal. 
We can restrict transform $W$ to be orthogonal. Then we will solve next problem:

$$W^*= \arg\min_W ||WX - Y||_F \text{, where: } W^TW = I$$

$$I \text{- identity matrix}$$

Instead of making yet another regression problem we can find optimal orthogonal transformation using singular value decomposition. It turns out that optimal transformation $W^*$ can be expressed via SVD components:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$
$$W^*=UV^T$$

In [15]:
def learn_transform(X_train, Y_train):
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in the formula
    """
    cross_cov = np.dot(X_train.T, Y_train)
    U, _, Vt = np.linalg.svd(cross_cov)
    W_star = np.dot(U, Vt)

    return W_star

In [16]:
W = learn_transform(X_train, Y_train)

In [17]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

[('апрель', 0.8237908482551575),
 ('сентябрь', 0.8049712181091309),
 ('март', 0.802565336227417),
 ('июнь', 0.8021841645240784),
 ('октябрь', 0.8001735806465149),
 ('ноябрь', 0.7934483289718628),
 ('февраль', 0.7914120554924011),
 ('июль', 0.7908107042312622),
 ('август', 0.7891014814376831),
 ('декабрь', 0.7686371803283691)]

In [18]:
assert precision(uk_ru_test, np.matmul(X_test, W)) >= 0.653
assert precision(uk_ru_test, np.matmul(X_test, W), 5) >= 0.824

## UK-RU Translator

Now we are ready to make simple word-based translator: for each word in source language in shared embedding space we find the nearest in target language.


In [19]:
with open("fairy_tale.txt", "r", encoding="utf-8") as inpf:
    uk_sentences = [line.rstrip().lower() for line in inpf]

In [20]:
import re

def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find Ukrainian embedding for each word in the sentence
    * transform Ukrainian embedding vector
    * find nearest Russian word and replace
    """
    uk_words = re.findall(r'\w+|[^\w\s]', sentence, re.UNICODE)
    
    translated_words = []
    
    for word in uk_words:
        if re.match(r'[\d\W_]+', word):
            translated_words.append(word)
            continue
        
        if word in uk_emb:
            uk_vector = uk_emb[word]
            transformed_vector = np.matmul(uk_vector, W)
            try:
                similar_words = ru_emb.most_similar([transformed_vector], topn=1)
                most_similar_ru_word = similar_words[0][0]
                translated_words.append(most_similar_ru_word)
            except KeyError:
                translated_words.append(word)
        else:
            translated_words.append(word)
    
    translation = " ".join(translated_words)
    return translation

In [21]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

In [22]:
for sentence in uk_sentences:
    print("src: {}\ndst: {}\n".format(sentence, translate(sentence)))

src: лисичка - сестричка і вовк - панібрат
dst: лисичка - сестричка и волк - панібрат

src: як була собі лисичка та зробила хатку, та й живе. а це приходять холоди. от лисичка замерзла та й побігла в село вогню добувать, щоб витопити. прибігає до одної баби та й каже:
dst: как была себе лисичка и сделала избушку , и и живет . а оно приходят морозы . из лисичка замерзла и и побежала во село огня добувать , чтобы витопити . прибегает к одной бабы и и говорит :

src: — здорові були, бабусю! з неділею... позичте мені огню, я вам одслужу.
dst: — здоровые были , бабушку ! со воскресеньем . . . позичте мне огня , мной тебе одслужу .

src: — добре, — каже, — лисичко - сестричко. сідай погрійся трохи, поки я пиріжечки повибираю з печі!
dst: — хорошо , — говорит , — лисичко - сестричка . садись погрійся немного , пока мной пирожки повибираю со печи !

src: а баба макові пиріжки пекла. от баба вибирає пиріжки та на столі кладе, щоб прохололи; а лисичка підгляділа та за пиріг, та з хати... виїла м

src: лягла на шляху та й лежить. ідуть чумаки з рибою; вона притаїлась, мов нежива. чумаки дивляться — аж лисиця:
dst: легла по пути и и лежит . идут казаки со рыбой ; она притаїлась , языки неживая . казаки смотрят — аж лисица :

src: — візьмім, — кажуть, — бра, та продамо — буде за що хоч погрітися!
dst: — візьмім , — дескать , — бра , и продадим — будет за что хоть погреться !

src: скинули її на останній віз та й поїхали. ідуть та й їдуть. а лисичка - сестричка бачить, що вони не дивляться, та все кида по рибці на дорогу, все кида... от як накидала вже багато, тоді нишком і сама злізла. чумаки ж поїхали собі далі, а вона позбирала рибку, сіла та й їсть.
dst: сбросили ей по последний визы и и поехали . идут и и едут . а лисичка - сестричка видит , что они не смотрят , и всё кида по рыбки по дорогу , всё кида . . . из как навязала уже много , тогда втихаря и сама слезла . казаки же поехали себе дальше , а она позбирала рыбку , присела и и ест .

src: коли це біжить вовчик:
dst: когда

Not so bad, right? We can easily improve translation using language model and not one but several nearest neighbours in shared embedding space. But next time.

## 2 Task)  7 points

###  Download embedding from here: https://fasttext.cc/docs/en/crawl-vectors.html (download text format). Depending on the group, choose the correct Slavic language embedding. After downloading and using the embedding, repeat all the tasks from first part, but you dont need set asserts in this task and generate the same simple text (at least 200 words) and translate it into Russian.


In [23]:
bg_emb = KeyedVectors.load_word2vec_format(r"cc.bg.300.vec")

In [27]:
import numpy as np

def load_word_pairs(filename):
    with open(filename, "r", encoding="utf-8") as inpf:
        pairs = [(bg, ru) for line in inpf if (bg := line.rstrip().split("\t")[0]) in bg_emb 
                 and (ru := line.rstrip().split("\t")[1]) in ru_emb]
    
    if not pairs:
        return [], np.empty((0, len(next(iter(bg_emb.values()))))), np.empty((0, len(next(iter(ru_emb.values())))))

    bg_ru_pairs = np.array(pairs, dtype=object)
    bg_vectors = np.array([bg_emb[bg] for bg, _ in pairs])
    ru_vectors = np.array([ru_emb[ru] for _, ru in pairs])
    
    return bg_ru_pairs.tolist(), bg_vectors, ru_vectors

In [28]:
bg_ru_train, X_train, Y_train = load_word_pairs("bg_rus.train.txt")

In [30]:
from sklearn.linear_model import LinearRegression

mapping = LinearRegression(fit_intercept=False)
mapping.fit(X_train, Y_train)

LinearRegression(fit_intercept=False)

In [31]:
august = mapping.predict(bg_emb["август"].reshape(1, -1))
ru_emb.most_similar(august)

[('июнь', 0.8433870673179626),
 ('сентябрь', 0.8430496454238892),
 ('апрель', 0.8423399925231934),
 ('ноябрь', 0.8406562805175781),
 ('октябрь', 0.8402948975563049),
 ('март', 0.8316455483436584),
 ('август', 0.8205991387367249),
 ('февраль', 0.8201203346252441),
 ('июль', 0.8180010318756104),
 ('декабрь', 0.8157303333282471)]

In [32]:
def learn_transform(X_train, Y_train):
    cross_cov = np.dot(X_train.T, Y_train)
    U, _, Vt = np.linalg.svd(cross_cov)
    W_star = np.dot(U, Vt)

    return W_star

In [33]:
W = learn_transform(X_train, Y_train)

In [34]:
ru_emb.most_similar([np.matmul(bg_emb["април"], W)])

[('июнь', 0.7917863726615906),
 ('март', 0.7865204811096191),
 ('апрель', 0.7818771004676819),
 ('ноябрь', 0.7782160639762878),
 ('октябрь', 0.7698571085929871),
 ('сентябрь', 0.7678281664848328),
 ('февраль', 0.7657983899116516),
 ('июль', 0.7602317929267883),
 ('декабрь', 0.7577673196792603),
 ('август', 0.7403132319450378)]

In [35]:
with open("golden_apple.txt", "r", encoding="utf-8") as inpf:
    bg_sentences = [line.rstrip().lower() for line in inpf]

In [36]:

def translate(sentence):
    bg_words = re.findall(r'\w+|[^\w\s]', sentence, re.UNICODE)
    
    translated_words = []
    
    for word in bg_words:
        if re.match(r'[\d\W_]+', word):
            translated_words.append(word)
            continue
        
        if word in bg_emb:
            bg_vector = bg_emb[word]
            
            transformed_vector = np.matmul(bg_vector, W)
            try:
                similar_words = ru_emb.most_similar([transformed_vector], topn=1)
                most_similar_ru_word = similar_words[0][0]
                translated_words.append(most_similar_ru_word)
            except KeyError:
                translated_words.append(word)
        else:
            translated_words.append(word)
    
    translation = " ".join(translated_words)
    return translation

In [37]:
sentence = "котка хвана мишка"
translated = translate(sentence)
print(f"Source: {sentence}")
print(f"Translated: {translated}")

Source: котка хвана мишка
Translated: кошка поймал мышка


In [38]:
for sentence in bg_sentences:
    print("src: {}\ndst: {}\n".format(sentence, translate(sentence)))

src: в едно малко село живеела млада и красива мома на име мария.
dst: в одно немного село жила женщина и красивая девица на имя мадонна .

src: тя била известна в селото със своята доброта и трудолюбие.
dst: то она известная в селом с собственная доброта и трудолюбие .

src: всеки ден тя помагала на старите хора и се грижела за животните в двора си.
dst: каждый день то медикаменты на старых людей и да служанка за животных в дворе да .

src: един ден, докато работела в градината си, тя забелязала нещо блестящо между храстите.
dst: один день , когда работала в сад да , то замечала нечто блестящее между куст .

src: когато се приближила, открила златно ябълко, което сияело като слънце.
dst: когда да пискнула , отыскала серебристое морковный , это сиянье как солнце .

src: мария го взела в ръцете си и усетила, че не е обикновено ябълко.
dst: мадонна да взяла в руках да и ощутила , что да то обычно морковный .

src: то било магическо. изведнъж се появил стар човек, който ѝ се усмихнал и ка

## Would you like to learn more?

### Articles:
* [Exploiting Similarities among Languages for Machine Translation](https://arxiv.org/pdf/1309.4168)  - entry point for multilingual embedding studies by Tomas Mikolov (the author of W2V)
* [Offline bilingual word vectors, orthogonal transformations and the inverted softmax](https://arxiv.org/pdf/1702.03859) - orthogonal transform for unsupervised MT
* [Word Translation Without Parallel Data](https://arxiv.org/pdf/1710.04087)
* [Loss in Translation: Learning Bilingual Word Mapping with a Retrieval Criterion](https://arxiv.org/pdf/1804.07745)
* [Unsupervised Alignment of Embeddings with Wasserstein Procrustes](https://arxiv.org/pdf/1805.11222)

### Repos (with ready-to-use multilingual embeddings):
* https://github.com/facebookresearch/MUSE

* https://github.com/Babylonpartners/fastText_multilingual -